<a href="https://colab.research.google.com/github/4zan4/My-Lab/blob/main/Login_test_common_Vulnerability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from urllib.parse import urlparse, urlencode


In [3]:
# Function to test the login page for common vulnerabilities
def test_login_page(url):
    try:
        # Check the initial response
        response = requests.get(url)
        if response.status_code != 200:
            return f"Failed to access the login page. Status code: {response.status_code}"

        potential_issues = []

        # Test for Open Redirect
        redirect_payload = {
            "redirect_uri": "https://evil.com"
        }
        parsed_url = urlparse(url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        modified_url = f"{base_url}?{urlencode(redirect_payload)}"
        redirect_response = requests.get(modified_url)

        if redirect_response.status_code == 302 and "evil.com" in redirect_response.headers.get("Location", ""):
            potential_issues.append("Open Redirect vulnerability detected.")

        # Check for improper CSRF protection
        if "state=" not in url or "nonce=" not in url:
            potential_issues.append("Potential CSRF protection issue: Missing state or nonce parameter.")

        # Test for Weak Code Challenge
        if "code_challenge=" in url and "code_challenge_method=S256" not in url:
            potential_issues.append("Potential Weak PKCE Implementation: Missing S256 code challenge method.")

        # Return findings
        if potential_issues:
            return potential_issues
        else:
            return "No common vulnerabilities found."

    except Exception as e:
        return f"An error occurred: {str(e)}"

In [ ]:
# Example usage
login_url = ""
results = test_login_page(login_url)
for issue in results:
    print(issue)